In [1]:
import os
import rasterio
import numpy as np
import xarray as xr

In [2]:
# Paths
input_dir = "/scratch/project_2000908/nousu/mNFI_to_NEP/MNFI/classified_mnfi_10x10_reproj/"
output_nc_file = "/scratch/project_2000908/nousu/mNFI_to_NEP/FINAL_LAYERS/mnfi_layers.nc"

# Get all .tif files
tif_files = sorted([f for f in os.listdir(input_dir) if f.endswith(".tif")])

# Initialize storage
data_vars = {}
coords = {}

# Read and process rasters
for i, fname in enumerate(tif_files):
    varname = os.path.splitext(fname)[0]  # Variable name from filename
    path = os.path.join(input_dir, fname)

    with rasterio.open(path) as src:
        data = src.read(1)
        nodata = src.nodata
        data = np.where(data == nodata, np.nan, data)

        # Only read coordinates once
        if i == 0:
            height, width = data.shape
            transform = src.transform
            x_coords = transform.c + transform.a * np.arange(width)
            y_coords = transform.f + transform.e * np.arange(height)
            coords["x"] = x_coords
            coords["y"] = y_coords[::-1]  # reverse Y axis for north-up

        # Add flipped data to ensure coordinate match
        data_vars[varname] = (("y", "x"), np.flipud(data))

# Create Dataset
ds = xr.Dataset(data_vars=data_vars, coords=coords)

# Add global metadata description
#ds.attrs["description"] = (
#    "mNFI variable names follow the pattern 'ika_X_Y_Z':\n"
#    " - X (1-3): land class (1 = forest, 2 = other wooded land, 3 = other land)\n"
#    " - Y (1 or 2): site main class (1 = mineral, 2 = peat)\n"
#    " - Z (1 or 2): site fertility  (1 = rich, 2 = poor)"
#)
ds.attrs["description"] = (
    "mNFI variable names follow the pattern 'ika_X':\n"
    " - X (1-12) is the classification of land class, site main class, site fertility combination"
)

# Save to NetCDF
ds.to_netcdf(output_nc_file)
ds.close()

print(f"Saved NetCDF to: {output_nc_file}")

Saved NetCDF to: /scratch/project_2000908/nousu/mNFI_to_NEP/FINAL_LAYERS/mnfi_layers.nc


In [3]:
# Paths
input_dir = "/scratch/project_2000908/nousu/mNFI_to_NEP/METEO/AVGS/"
output_nc_file = "/scratch/project_2000908/nousu/mNFI_to_NEP/FINAL_LAYERS/meteo_layers.nc"

# Get all .tif files
tif_files = sorted([f for f in os.listdir(input_dir) if f.endswith(".tif")])

# Initialize storage
data_vars = {}
coords = {}

# Read and process rasters
for i, fname in enumerate(tif_files):
    varname = os.path.splitext(fname)[0]  # Variable name from filename
    path = os.path.join(input_dir, fname)

    with rasterio.open(path) as src:
        data = src.read(1)
        nodata = src.nodata
        data = np.where(data == nodata, np.nan, data)

        # Only read coordinates once
        if i == 0:
            height, width = data.shape
            transform = src.transform
            x_coords = transform.c + transform.a * np.arange(width)
            y_coords = transform.f + transform.e * np.arange(height)
            coords["x"] = x_coords
            coords["y"] = y_coords[::-1]  # reverse Y axis for north-up

        # Add flipped data to ensure coordinate match
        data_vars[varname] = (("y", "x"), np.flipud(data))

# Create Dataset
ds = xr.Dataset(data_vars=data_vars, coords=coords)

# Add global metadata description
ds['vapor_pressure_deficit'].attrs = {'units': 'kPa', 'description': 'average growing season water vapor pressure deficit (esa - PH2O_kpa)'}
ds['air_temperature'].attrs = {'units': 'degree_Celsius', 'description': 'average temperature'}
ds['growing_season_length'].attrs = {'units': 'number of days', 'description': 'average growing season length'}

# Save to NetCDF
ds.to_netcdf(output_nc_file)
ds.close()

print(f"Saved NetCDF to: {output_nc_file}")

Saved NetCDF to: /scratch/project_2000908/nousu/mNFI_to_NEP/FINAL_LAYERS/meteo_layers.nc


In [2]:
# Paths
tif_file = r"/scratch/project_2000908/nousu/mNFI_to_NEP/MASKS/khk_mask_tm35.tif"
output_nc_file = "/scratch/project_2000908/nousu/mNFI_to_NEP/FINAL_LAYERS/regional_mask.nc"

# Initialize storage
data_vars = {}
coords = {}

# Get variable name from the filename
varname = 'region'

with rasterio.open(tif_file) as src:
    data = src.read(1)
    nodata = src.nodata
    data = np.where(data == nodata, np.nan, data)

    height, width = data.shape
    transform = src.transform
    x_coords = transform.c + transform.a * np.arange(width)
    y_coords = transform.f + transform.e * np.arange(height)
    coords["x"] = x_coords
    coords["y"] = y_coords[::-1]  # reverse Y axis for north-up

    # Add flipped data to ensure coordinate match
    data_vars[varname] = (("y", "x"), np.flipud(data))

# Create Dataset
ds = xr.Dataset(data_vars=data_vars, coords=coords)

# Save to NetCDF
ds.to_netcdf(output_nc_file)
ds.close()

print(f"Saved NetCDF to: {output_nc_file}")

Saved NetCDF to: /scratch/project_2000908/nousu/mNFI_to_NEP/FINAL_LAYERS/regional_mask.nc
